# Пример решения кейса «VK: ScoreWorker»

Чтобы вам было легче приступить к работе, мы подготовили пример анализа отзывов о сотруднике на основе обратной связи от коллег и руководителей. **Это не финальное решение, и некоторые важные шаги упущены**, но оно поможет понять основную идею задачи.

## Шаг 0
**Выбор данных для анализа**

Возьмём случайный пример из набора данных:

```json
[
    {
        "ID_reviewer": 4624,
        "ID_under_review": 6135,
        "review": "К [ИМЯ] можно прийти по любому вопросу: [КОМПАНИЯ]/[КОМПАНИЯ] - отличительная черта: вежливость, мягкость, интеллигентность (не в ущерб отстаиваемым интересам) и при этом высокий уровень профессионализма.\nВсегда получаю удовольствие от взаимодействия с [ИМЯ]: ни один из запросов не остался без ответа - всегда есть логическое объяснение почему да или почему нет."
    },
    {
        "ID_reviewer": 325,
        "ID_under_review": 6135,
        "review": "Хороший специалист"
    },
    {
        "ID_reviewer": 28,
        "ID_under_review": 6135,
        "review": "хотелось бы больше ориентации в сторону сделать дело, видимо исторически тяготеешь как то в сторону не пущать. ну или так мне показалось"
    },
    {
        "ID_reviewer": 59073,
        "ID_under_review": 6135,
        "review": "[ИМЯ], спасибо за конструктивное взаимодействие, командную работу и нацеленность на результат, открытость к диалогу и поиск эффективных решений"
    },
    {
        "ID_reviewer": 475,
        "ID_under_review": 6135,
        "review": "лучший программист эвер! и классный человек! однозначно недополучает, нужно поднимать зп!"
    }
]
```

В этом примере ID сотрудника, о котором оставлены отзывы — 6135, его и проанализируем.

## Шаг 1
**Подготовка данных для анализа**

Для начала, мы подготовили данные для анализа, извлекая отзывы о сотруднике с ID 6135.

In [ ]:
import json

def load_reviews(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reviews = json.load(file)
    return reviews

file_path = "sample_reviews.json"
reviews = load_reviews(file_path)

## Шаг 2
**Формирование запроса для LLM**

Теперь сформируем запрос для модели `LLama3.1-Instruct`. Далее, с её помощью мы оценим сотрудника на основе выбранных отзывов. Запрос содержит тексты отзывов.

In [ ]:
def prepare_prompt(reviews):
    prompt = "Here are some reviews about an employee:\n\n"
    for i, review in enumerate(reviews, start=1):
        prompt += f"Review {i}:\n{review['review']}\n\n"

    prompt += "Based on these reviews, evaluate the employee on a scale from 1 to 5 for the following criteria:\n"
    prompt += "1. Professionalism\n2. Teamwork\n3. Communication\n4. Initiative\n5. Overall Performance\n"
    prompt += "Add short (5 sentences) explanation for each score you assigned."
    return prompt

## Шаг 3
**Вызов API модели LLM**

Мы отправили подготовленный запрос в API модели LLM, чтобы получить оценку по каждому критерию и общую оценку сотрудника.

In [ ]:
import requests
import json

def evaluate_reviews_with_llm(prompt):
    url = "https://vk-scoreworker-case-backup.olymp.innopolis.university/generate"
    data = {
        "prompt": [prompt],
        "apply_chat_template": True,
        "system_prompt": "You are a helpful assistant.",
        "max_tokens": 400,
        "n": 1,
        "temperature": 0.7
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

## Результат
Модель оценила сотрудника и выдала следующий ответ:

In [ ]:
# Prepare the prompt for the LLM
prompt = prepare_prompt(reviews)

# Send the prompt to the LLM and get the evaluation
evaluation = evaluate_reviews_with_llm(prompt)

# Output the result
print("LLM Evaluation of Employee based on Reviews:")
print(evaluation)

LLM Evaluation of Employee based on Reviews:
Based on the reviews, I have evaluated the employee as follows:

1. Professionalism: 5
The employee demonstrates high professionalism, as stated in Review 1, having a "высокий уровень профессионализма" (high level of professionalism). Review 2 simply calls the employee a "Хороший специалист" (good specialist), implying competence in their field. Review 4 also mentions the employee being focused on results and seeking effective solutions, showcasing professionalism. Review 5 praises the employee as the "лучший программист эвер" (best programmer ever), further solidifying their professionalism. Overall, the employee has consistently demonstrated expertise and dedication to their work.

2. Teamwork: 4
Review 4 highlights the employee's "командную работу" (teamwork), indicating their ability to collaborate with others. Review 1 also mentions the employee's politeness and softness, traits beneficial for effective teamwork. However, Review 3 sugge

## Следующие шаги
Данное решение учитывает только текст отзывов. Для более глубокого анализа можно рассмотреть дополнительные факторы, такие как частота отзывов, их позитивность/негативность, а также взаимосвязь между отзывами от разных коллег.

**Учтите, что**  
Решение участников должно составлять краткую сводку о сотруднике, включая числовые оценки по каждому критерию и общую оценку. Помните, что цель этого задания - создать инструмент, способный значительно ускорить и автоматизировать анализ обратной связи о сотрудниках.

Удачи в разработке решений!